In [13]:
!pip install dask[complete] modin[all] ray pandas pyyaml --quiet

In [14]:
import os
import pandas as pd
import dask.dataframe as dd
import modin.pandas as mpd
import ray
import yaml
import re
from datetime import datetime

# Start Ray manually
ray.shutdown()
ray.init(runtime_env={"env_vars": {"__MODIN_AUTOIMPORT_PANDAS__": "1"}})

2025-04-09 22:25:02,066	INFO worker.py:1852 -- Started a local Ray instance.


Python version:,3.11.11
Ray version:,2.44.1


In [4]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet

--2025-04-09 22:17:05--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.163.157.7, 3.163.157.72, 3.163.157.133, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.163.157.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47673370 (45M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-01.parquet.1’

yellow_tripdata_202 100%[===================>]  45.46M   234MB/s    in 0.2s    

2025-04-09 22:17:06 (234 MB/s) - ‘yellow_tripdata_2023-01.parquet.1’ saved [47673370/47673370]



In [6]:
import pandas as pd

df = pd.read_parquet("yellow_tripdata_2023-01.parquet")
df.to_csv("yellow_tripdata_2023-01.csv", index=False)

In [8]:
import time

start = time.time()
df_pandas = pd.read_csv("yellow_tripdata_2023-01.csv")
end = time.time()

print("Pandas read time:", end - start)

<ipython-input-8-a450b5a73dbe>:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pandas = pd.read_csv("yellow_tripdata_2023-01.csv")


Pandas read time: 15.85444688796997


In [18]:
import dask.dataframe as dd

start = time.time()
df_dask = dd.read_csv("yellow_tripdata_2023-01.csv")
df_dask.head()  # triggers computation
end = time.time()

print("Dask read time:", end - start)

Dask read time: 4.730542898178101


In [19]:
import modin.config as modin_cfg
modin_cfg.Engine.put("dask")  # use Dask instead of Ray

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:42585
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:45753'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:41757'
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:36495 name: 0
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:36495
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:48274
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:43309 name: 1
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:43309
INFO:distributed.core:Starting established connection to tcp://127

In [22]:
import modin.pandas as mpd
start = datetime.now()
df_modin = mpd.read_csv("yellow_tripdata_2023-01.csv")
end = datetime.now()
print("Modin (Dask backend) read complete. Time taken:", end - start)

Modin (Dask backend) read complete. Time taken: 0:00:25.799951


In [24]:
def clean_column_names(df):
    df.columns = [
        re.sub(r'\W+', '_', col.strip()).lower()
        for col in df.columns
    ]
    return df

df_clean = clean_column_names(df_pandas.copy())


In [26]:
schema = {
    'separator': '|',
    'columns': df_clean.columns.tolist()
}

with open('schema.yaml', 'w') as f:
    yaml.dump(schema, f)

# Display schema
!cat schema.yaml

columns:
- vendorid
- tpep_pickup_datetime
- tpep_dropoff_datetime
- passenger_count
- trip_distance
- ratecodeid
- store_and_fwd_flag
- pulocationid
- dolocationid
- payment_type
- fare_amount
- extra
- mta_tax
- tip_amount
- tolls_amount
- improvement_surcharge
- total_amount
- congestion_surcharge
- airport_fee
separator: '|'


In [28]:
with open('schema.yaml') as f:
    schema_loaded = yaml.safe_load(f)

expected_columns = schema_loaded['columns']
expected_separator = schema_loaded['separator']

# Validate column count and names
assert df_clean.columns.tolist() == expected_columns, "Column names mismatch"
print("Schema validated successfully!")

Schema validated successfully!


In [29]:
df_clean.to_csv("cleaned_output.txt.gz", sep='|', index=False, compression='gzip')

In [30]:
rows, cols = df_clean.shape
file_size = os.path.getsize("cleaned_output.txt.gz") / (1024 ** 2)

print("FILE SUMMARY:")
print(f"Total Rows: {rows}")
print(f"Total Columns: {cols}")
print(f"Output File Size: {file_size:.2f} MB")


FILE SUMMARY:
Total Rows: 3066766
Total Columns: 19
Output File Size: 53.37 MB
